In [1]:
import librosa
import pickle
import IPython.display as ipd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
data = pd.read_csv('kaggle/DATASET-balanced.csv')

print(data.head())


   chroma_stft       rms  spectral_centroid  spectral_bandwidth      rolloff  \
0     0.338055  0.027948        2842.948867         4322.916759  6570.586186   
1     0.443766  0.037838        2336.129597         3445.777044  3764.949874   
2     0.302528  0.056578        2692.988386         2861.133180  4716.610271   
3     0.319933  0.031504        2241.665382         3503.766175  3798.641521   
4     0.420055  0.016158        2526.069123         3102.659519  5025.077899   

   zero_crossing_rate       mfcc1       mfcc2      mfcc3      mfcc4  ...  \
0            0.041050 -462.169586   90.311272  19.073769  24.046888  ...   
1            0.047730 -409.413422  120.348808  -7.161531   5.114784  ...   
2            0.080342 -318.996033  120.490273 -24.625771  23.891073  ...   
3            0.047180 -404.636749  136.320908   2.308172  -3.907071  ...   
4            0.051905 -410.497925  152.731400 -18.266771  51.993462  ...   

     mfcc12    mfcc13    mfcc14    mfcc15    mfcc16    mfcc17 

In [3]:

X = data.iloc[:, :-1]  # Features
y = data.iloc[:, -1]   # Labels


label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
model = xgb.XGBClassifier(objective='binary:logistic', seed=42)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [6]:

y_pred = model.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

print(classification_report(y_test, y_pred))


print(confusion_matrix(y_test, y_pred))


Accuracy: 99.32%
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      1193
           1       0.99      1.00      0.99      1163

    accuracy                           0.99      2356
   macro avg       0.99      0.99      0.99      2356
weighted avg       0.99      0.99      0.99      2356

[[1182   11]
 [   5 1158]]


In [7]:
with open('XGBoost_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [8]:
AudioFromUser = 'data/for-norm/for-norm/testing/fake/file1.wav_16k.wav_norm.wav_mono.wav_silence.wav'

In [9]:
x, sr = librosa.load(AudioFromUser)

In [10]:
ipd.display(ipd.Audio(x, rate=sr))

In [11]:


def extract_audio_features(segment, sr, save_path=None):
    chroma_stft = np.mean(librosa.feature.chroma_stft(y=segment, sr=sr))
    rms = np.mean(librosa.feature.rms(y=segment))
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=segment, sr=sr)[0])
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=segment, sr=sr)[0])
    rolloff = np.mean(librosa.feature.spectral_rolloff(y=segment, sr=sr)[0])
    zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=segment)[0])
    mfccs = librosa.feature.mfcc(y=segment, sr=sr, n_mfcc=20)

    features = {
        'chroma_stft': chroma_stft,
        'rms': rms,
        'spectral_centroid': spectral_centroid,
        'spectral_bandwidth': spectral_bandwidth,
        'rolloff': rolloff,
        'zero_crossing_rate': zero_crossing_rate
    }

    for i in range(1, 21):
        features[f'mfcc{i}'] = np.mean(mfccs[i-1])
    return features


In [12]:
# Assuming 'segment' and 'sr' are defined elsewhere
audio_features = extract_audio_features(x, sr)


In [14]:

# Load the trained XGBoost model
with open('XGBoost_model.pkl', 'rb') as f:
    model = pickle.load(f)


# Prepare the features for prediction
feature_values = list(audio_features.values())
audio_features_array = np.array(feature_values).reshape(1, -1)  # Reshape to a single sample

# Make predictions directly using model.predict() with the NumPy array
prediction = model.predict(audio_features_array)

print(prediction)


[1]
